In [ ]:
import numpy as np 
import pandas as pd

import os
print(os.listdir("../input/new-york-city-taxi-fare-prediction"))
print(os.listdir("../input/nyctfp"))
#print(os.listdir())

In [ ]:
%%time
train = pd.read_csv("../input/nyctfp/train_010.csv", nrows = 3_000_000) #設定要train的筆數
test = pd.read_csv("../input/nyctfp/test_010.csv")

#train = pd.read_csv("train_010.csv", nrows = 2_000_000) #設定要train的筆數
#test = pd.read_csv("test.csv")

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.dtypes

In [ ]:
test.dtypes

In [ ]:
train.isnull().sum()

# Natural Constraint

In [ ]:
train[train['passenger_count']>6] #檢查乘客大於6的，再把他drop掉

In [ ]:
train = train[train['passenger_count']<=6] #檢查乘客大於6的，再把他drop掉

In [ ]:
max(train['passenger_count'])

In [ ]:
max(test['passenger_count'])

In [ ]:
print(test['pickup_longitude'].max(), test['pickup_longitude'].min(), 
      test['pickup_latitude'].max(), test['pickup_latitude'].min())

In [ ]:
print(test['dropoff_longitude'].max(), test['dropoff_longitude'].min(), 
      test['dropoff_latitude'].max(), test['dropoff_latitude'].min())

In [ ]:
train = train[train.pickup_longitude<=-72.986532]
train = train[train.pickup_longitude>=-74.252193]
train = train[train.pickup_latitude<=41.709555]
train = train[train.pickup_latitude>=40.573143]

In [ ]:
train = train[train.dropoff_longitude<=-72.990963]
train = train[train.dropoff_longitude>=-74.263242]
train = train[train.dropoff_latitude<=41.696683]
train = train[train.dropoff_latitude>=40.568973]

# Y

In [ ]:
'''
import re
date = []
# [0:3000000]
# [3000000:5000000]
# [5000000:8000000]
# [8000000:10000000]
for i in test['pickup_datetime']:
    ii = re.findall(r"\d+\-\d+\-\d+", i) #get the int in the pickup_datetime with ' yyyy-mm-dd '
    a = ''.join(str(ii)) #change list to string
    b = a.split() #strip with space
    date.append(b)
    
#check
#print(len(date))
#print(date[0:5])

#get items from year, putting them into list
date1 = []
for i in date:
    ii = ''.join(str(i))
    b = re.sub(r"\[*\"*\[*\'*\'*\]*\"*\]*",'', ii)
    date1.append(b)

#check
#print(date1[0:5])

#get month from date, putting them into list
month = []
for i in date:
    ii = ''.join(str(i))
    b = ii.split('-')
    month.append(b[1])

#check
#print(month[0:5])

time = []
# [0:3000000]
# [3000000:5000000]
# [5000000:8000000]
# [8000000:10000000]
for i in test['pickup_datetime']:
    ii = re.findall(r"\d+\:\d+\:\d+", i) #get the int in the pickup_datetime with ' hh:mm:ss'
    a = ''.join(str(ii)) #change list to string
    b = a.split() #strip [('')]
    time.append(b)

#check
#print(time[0:5])

#get item from time, putting into list
time1 = []
for i in time:
    ii = ''.join(str(i))
    b = re.sub(r"\[*\"*\[*\'*\'*\]*\"*\]*",'', ii)
    time1.append(b)

#check
#print(time1[0:5])

#build list for week
import calendar

y = []
m = []
d = []

for i in date1:
    ii = ''.join(str(i))
    b = ii.split('-')
    y.append(int(b[0]))
    m.append(int(b[1]))
    d.append(int(b[2]))
    
# convert date to week
days  =["1","2","3","4","5","6","7"]
week = []
for i, e, c in zip(y, m, d):
    a = calendar.weekday(i,e,c)
    week.append(days[a])
    
#get each hour in list time1
hour = []
for i in time1:
    ii = ''.join(str(i))
    b = ii.split(':')
    hour.append(int(b[0]))

#check
#print(hour[0:10])

# build time interval
for n, i in enumerate(hour):
    if  6<=i<16 :
        hour[n] = 1
    elif 16<=i<20 :
        hour[n] = 2
    else:
        hour[n] = 3
        
#check
#print(hour[0:10])

#build weekend
weekend = []

for i in week:
    if  i==6 :
        weekend.append(1)
    elif i==7:
        weekend.append(1)
    else:
        weekend.append(0)
        
import holidays
#the holidays in 'NY'
us_holidays = holidays.CountryHoliday('US', prov=None, state='NY')
holiday = []
for i in date1:
    if i in us_holidays:
        holiday.append(1)
    else:
        holiday.append(0)

# lat & long
pick_long = test.iloc[:,2].values
pick_lat = test.iloc[:,3].values
drop_long = test.iloc[:,4].values
drop_lat = test.iloc[:,5].values

from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2): # 经度1，纬度1，经度2，纬度2 （十进制度数）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # 将十进制度数转化为弧度
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # 地球平均半径，单位为公里
    return c * r * 1000
    
#test
import reverse_geocoder as rg #記得裝套件

train_pickup = test[['pickup_latitude', 'pickup_longitude']]
train_dropoff = test[['dropoff_latitude', 'dropoff_longitude']]

pickup_place = [] #array
dropoff_place = []

for i, j in train_pickup.iterrows():
    pickup_place.append((j["pickup_latitude"], j["pickup_longitude"]))
    
for i, j in train_dropoff.iterrows():
    dropoff_place.append((j["dropoff_latitude"], j["dropoff_longitude"]))

pickup_results = rg.search(pickup_place)
dropoff_results = rg.search(dropoff_place)

pur = [pickup_results[i]['name'] for i in range(0, len(pickup_results))]
dor = [dropoff_results[i]['name'] for i in range(0, len(dropoff_results))]

#append columns to test
test['date'] = date1
test['month'] = month
test['time'] = time1
test['week'] = week
test['time_interval'] = hour
test['weekend'] = weekend
test['holiday'] = holiday
test['pickup_place'] = pd.Series(pur)
test['dropoff_place'] = pd.Series(dor)

##linear_distance
pick = test.iloc[:,14].values
drop = test.iloc[:,15].values
area1 = []
area2 = []
area3 = []
area4 = []
area5 = []

for n in range(9914):
    if test['pickup_place'][n] == 'New York City' and test['dropoff_place'][n] != 'New York City' :
        area1.append(1)
    elif test['pickup_place'][n] != 'New York City' and test['dropoff_place'][n] == 'New York City' :
        area1.append(1)
    else:
        area1.append(0)
        
#buid linear_distance
linear_distance = []
for i in range(0,9914):
    linear_distance.append(haversine(pick_long[i], pick_lat[i], drop_long[i], drop_lat[i]))

test['area_crossing'] = area1
test['linear_distance'] = linear_distance
'''

In [ ]:
'''
train['pickup_place'] = train['pickup_place'].astype('category').values
train['dropoff_place'] = train['pickup_place'].astype('category').values

train['passenger_count'] = train['passenger_count'].astype('float64').values
test['passenger_count'] = test['passenger_count'].astype('float64').values

train['date'] = train['date'].astype('category').values
test['date'] = test['date'].astype('category').values

train['month'] = train['month'].astype('category').values
test['month'] = test['month'].astype('category').values
train['week'] = train['week'].astype('category').values
test['week'] = test['week'].astype('category').values

test['pickup_place'] = test['pickup_place'].astype('category').values
test['dropoff_place'] = test['dropoff_place'].astype('category').values
'''

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.dtypes

In [ ]:
test.dtypes

# Model

In [ ]:
train.columns

In [ ]:
features = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude',
            'passenger_count', 'month', 'week', 'time_interval', 'weekend', 'holiday', 
            'pickup_place', 'dropoff_place', 'area_crossing', 
            'linear_distance']
target = ['fare_amount']

In [ ]:
X_features = train.iloc[:, :][features]
X_target = train.iloc[:, :][target]

y_features = test.iloc[:, :][features]

In [ ]:
X_features.head()

https://tech.yandex.com/catboost/doc/dg/concepts/algorithm-main-stages_cat-to-numberic-docpage/

In [ ]:
categorical_features_indices = np.where(X_features.dtypes != np.float)[0]

In [ ]:
categorical_features_indices

## Data Splitting

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_validation, y_train, y_validation = train_test_split(X_features, X_target, test_size=0.3, random_state=0)
X_test = y_features

# CatBoost模型

In [ ]:
from catboost import CatBoostRegressor, Pool, cv
from sklearn.metrics import accuracy_score

In [ ]:
train_pool = Pool(X_train, y_train, cat_features=categorical_features_indices)
validate_pool = Pool(X_validation, y_validation, cat_features=categorical_features_indices)

In [ ]:
#default parameters
model = CatBoostRegressor(
    num_boost_round=700,
    learning_rate=0.4, #default 0.03
    depth=10, #default 6
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=0,
#    logging_level='Verbose',
    use_best_model=True,
    od_type='IncToDec',
    od_pval=0.01,
    od_wait=40
)

In [ ]:
%%time
model.fit(
    X_train, y_train,
    cat_features=categorical_features_indices,
    eval_set=validate_pool,
#    logging_level='Verbose',
    plot=True #可看圖!!!
);

In [ ]:
'''
cv_params = model.get_params() 
cv_params.update({
    'loss_function': 'RSME' #改loss_function
})

#進行cross validation，採用別的loss function
cv_data = cv(
    train_pool,
    cv_params,
    plot=True
) 
'''

https://tech.yandex.com/catboost/doc/dg/concepts/loss-functions-docpage/#loss-functions

In [ ]:
y_pred_cb = model.predict(X_test)

In [ ]:
feature_importances = model.get_feature_importance(train_pool)

feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
submission = pd.DataFrame()
submission['key'] = test.key
submission['fare_amount'] = y_pred_cb
submission.to_csv('submission_cb010.csv', index = False)

In [ ]:
ss = pd.read_csv('../input/new-york-city-taxi-fare-prediction/sample_submission.csv')
#ss = pd.read_csv('sample_submission.csv')
ss.loc[:, 'fare_amount'] = y_pred_cb

ss.to_csv('submission_cb010.csv', index=False)